<a href="https://colab.research.google.com/github/PondKann/CXR-Project/blob/main/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###### import

In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import pandas as pd
from torchsummary import summary
from skimage.io import imread, imsave
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from imutils import paths
import argparse

#### Load data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/train'
validation_dir = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/test'

In [5]:
train_normal_dir = os.path.join(train_dir,'NORMAL')
train_pneumonia_dir = os.path.join(train_dir,'PNEUMONIA')
validation_normal_dir = os.path.join(validation_dir,'NORMAL')
validation_pneumonia_dir = os.path.join(validation_dir,'PNEUMONIA')

#### สร้างตารางเก็บชื่อภาพ และโรค 2
ข้อมูลในไฟล์ train ทั้ง normal และ pneunomia มารวมอยู่ในไฟล์ all_train แล้วลองนำมาทำ data augment

In [6]:
train_normal_dir = os.path.join(train_dir,'NORMAL')
train_pneumonia_dir = os.path.join(train_dir,'PNEUMONIA')

In [7]:
all_train = '/content/drive/My Drive/Senior Project/xray_dataset_covid19/all_train'

สร้าง data dict ของรูป

In [8]:
normal = []
for i in range(len(os.listdir(train_normal_dir))):
    img = os.listdir(train_normal_dir)[i]
    normal.append(img)
tnormal = pd.DataFrame({'ImageName':[normal][0],
                                       'Type1': "Normal" ,
                                       'Type2': 0                      })

In [9]:
pneumonia = []
for i in range(len(os.listdir(train_pneumonia_dir))):
    img = os.listdir(train_pneumonia_dir)[i]
    pneumonia.append(img)
tpneumonia = pd.DataFrame({'ImageName':[pneumonia][0],
                                       'Type1': "Pneumonia" ,
                                       'Type2': 1                        })

In [10]:
data_train = pd.concat([tnormal, tpneumonia])
data_train.head(3)

,ImageName,Type1,Type2
0,IM-0006-0001.jpeg,Normal,0
1,IM-0003-0001.jpeg,Normal,0
2,IM-0001-0001.jpeg,Normal,0


# แบบที่ 1 

##แบ่งข้อมูล

In [11]:
train_img = []
for ImageName in tqdm(data_train['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = all_train +'/' + ImageName                       # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))
    train_img.append(img)


100%|██████████| 148/148 [01:16<00:00,  1.93it/s]


In [12]:
data = np.array(train_img, dtype="float32") / 255.0

In [13]:
#1
labels = np.array(data_train[['Type1']] )           #shape (148, 1) ค่าเป็น 0. กับ 1.  รัน model2.fit  ได้

len(labels)
#labels.shape = (len(labels),)                          #เปลี่ยน dimention เป็น 1 มิติ 
labels.shape

(148, 1)

In [14]:
le = LabelEncoder()                                                                   #จะแปลงชื่อ class เป็นตัวเลข
labels = le.fit_transform(labels)                                                  # dimention ของ Y จะเป็น 1 (37,) ต้องเปลี่ยนเป็น (37,1) ก่อน ถึงจะรัน report ได้

labels.shape = (len(labels),1)
labels.shape

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(148, 1)

In [15]:
le.classes_

array(['Normal', 'Pneumonia'], dtype=object)

In [16]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

In [17]:
trainX.ndim

4

## Data Augmentation  #1

### train

In [18]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [19]:
# initialize an our data augmenter as an "empty" image data generator
aug = ImageDataGenerator()

In [26]:
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.25,
		width_shift_range=0.25,
		height_shift_range=0.25,
		shear_range=0.25,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [21]:
base_model = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

74850304/74836368 [==============================] - 1s 0us/step


In [22]:
base_model.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(average_pooling_layer)   #  unit = 1 คือ มิติของ output เป็น 1 มีค่าความน่าจะเป็นค่าเดียว
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [23]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [28]:
# train the network

H = model.fit(
	x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=5)

Epoch 1/5
13/13 [==============================] - 29s 2s/step - loss: 0.5366 - accuracy: 0.7961 - val_loss: 0.5332 - val_accuracy: 0.8649
Epoch 2/5
13/13 [==============================] - 29s 2s/step - loss: 0.5144 - accuracy: 0.8252 - val_loss: 0.5204 - val_accuracy: 0.8649
Epoch 3/5
13/13 [==============================] - 29s 2s/step - loss: 0.5114 - accuracy: 0.8058 - val_loss: 0.5078 - val_accuracy: 0.8919
Epoch 4/5
13/13 [==============================] - 29s 2s/step - loss: 0.4781 - accuracy: 0.8544 - val_loss: 0.4966 - val_accuracy: 0.8919
Epoch 5/5
13/13 [==============================] - 31s 2s/step - loss: 0.4690 - accuracy: 0.8350 - val_loss: 0.4852 - val_accuracy: 0.9189


### test

In [29]:
predictions = model.predict(testX, batch_size=BS)

In [30]:
predictions1 = []
for i in tqdm(predictions):
    if i > 0.5:
        predictions1.append(1)
    else:
        predictions1.append(0)
predictions1 = np.array(predictions1, dtype="float32")
predictions1.shape = (len(predictions1) ,1)                         #แปลงให้ predictions2 เป็น 2 มิติ
predictions1.ndim

100%|██████████| 37/37 [00:00<00:00, 38845.87it/s]


2

In [31]:
predictions1.max(axis=1)

array([0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       0., 1., 1.], dtype=float32)

In [32]:
testY.max(axis=1)

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1])

In [33]:
print(classification_report(testY.max(axis=1),	
                            predictions1.max(axis=1),  
							target_names=le.classes_ ))

              precision    recall  f1-score   support

      Normal       0.87      1.00      0.93        20
   Pneumonia       1.00      0.82      0.90        17

    accuracy                           0.92        37
   macro avg       0.93      0.91      0.92        37
weighted avg       0.93      0.92      0.92        37



## Data Augmentation  #2

In [34]:
labels = np.array(data_train[['Type1']] )           #shape (148, 1) ค่าเป็น 0. กับ 1.  รัน model2.fit  ได้

In [35]:
#2
le = LabelEncoder()                                                                   #จะแปลงชื่อ class เป็นตัวเลข
labels = le.fit_transform(labels)                                                  # dimention ของ Y จะเป็น 1 (37,) ต้องเปลี่ยนเป็น (37,1) ก่อน ถึงจะรัน report ได้
labels = to_categorical(labels, num_classes=2)                            # แปลงเป็นเลข ดัมมี่ (1เป็น 1,0 ----- 0 เป็น  0,1)    
            #shape (148, 2) ค่าเดิม 0. กับ 1.  หลังจากรันจะเป็น [0., 1.] กับ [1., 0.]  ------->  แต่จะรัน model2.fit ไม่ได้ อาจเพราะเป็นโมเดลสำหรับ output แค่ 1 ค่า 

(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### train

In [36]:
# initialize the initial learning rate, batch size, and number of
# epochs to train for
INIT_LR = 1e-1
BS = 8
EPOCHS = 50

In [37]:
# initialize an our data augmenter as an "empty" image data generator
aug = ImageDataGenerator()

In [43]:
aug = ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.25,
		width_shift_range=0.25,
		height_shift_range=0.25,
		shear_range=0.25,
		horizontal_flip=True,
		fill_mode="nearest")

DenseNet201

In [39]:
base_model2 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [40]:
base_model2.trainable=False

#Add custom head
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model2.output)
prediction_layer = tf.keras.layers.Dense(units=2, activation="sigmoid")(average_pooling_layer)   # ลองแก้ unit จาก 1 เป็น 2
model2 = tf.keras.models.Model(inputs=base_model2.input, outputs=prediction_layer)

In [41]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [44]:
# train the network

H = model2.fit(
	x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=2)

Epoch 1/2
13/13 [==============================] - 32s 3s/step - loss: 0.7104 - accuracy: 0.5631 - val_loss: 0.6772 - val_accuracy: 0.6216
Epoch 2/2
13/13 [==============================] - 31s 2s/step - loss: 0.6581 - accuracy: 0.6311 - val_loss: 0.6596 - val_accuracy: 0.6216


### test

In [ ]:
predictions = model2.predict(testX, batch_size=BS)

In [ ]:
predictions.ndim

2

In [ ]:
predictions[:5]

array([[0.6088672 , 0.5897096 ],
       [0.5205667 , 0.66411275],
       [0.5682447 , 0.49947655],
       [0.5207887 , 0.58778536],
       [0.41414273, 0.4894217 ]], dtype=float32)

In [ ]:
print(classification_report(testY.max(axis=1),	
                            predictions.max(axis=1),  
							target_names=le.classes_ ))

ValueError: ignored

## Fine tune

In [ ]:
len(base_model2.layers)

707

In [ ]:
base_model2.trainable = True
for layer in base_model2.layers[:500]:    #fix w & bias in layer 0-500
    layer.trainable = False

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
#train
model2.fit_generator(train_generator, epochs=10, 
                    validation_data=valid_generator,
                    callbacks=[callback2])

# แบบที่ 2

##แบ่งข้อมูล

In [ ]:
train_img = []
for ImageName in tqdm(data_train['ImageName']):               # รัน ชื่อรูป ในไฟล์ data_test2
    image_path = all_train +'/' + ImageName                       # ดึงรูปในโฟล์เดอร์มาทีละรูป
    img = cv2.imread(image_path)                                            # อ่านรูป
    img = cv2.resize(img, (224, 224))
    train_img.append(img)


100%|██████████| 148/148 [00:06<00:00, 23.11it/s]


In [ ]:
data = np.array(train_img, dtype="float32") / 255.0

In [ ]:
labels = np.array(data_train[['Type1']])          

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
#อิมพอร์ต LabelBinarizer ของ sklearn มาใช้
labels = lb.fit_transform(labels)
#เรียกใช้ฟังก์ชั่น fit_transform เพื่อแปลง labels เป็นเลขฐานสอง

In [ ]:
labels[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [ ]:
lb.classes_

array(['Normal', 'Pneumonia'], dtype='<U9')

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)

In [ ]:
base_model2 = tf.keras.applications.DenseNet201(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [ ]:
base_model2.trainable=False

In [ ]:
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model2.output)
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(average_pooling_layer)

In [ ]:
model2 = tf.keras.models.Model(inputs=base_model2.input, outputs=prediction_layer)

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
callback2 = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2,
    mode="max")

In [ ]:
(trainX, testX, trainY, testY) 

In [ ]:
model2.fit(trainX, trainY, epochs=3 )

Epoch 1/3
4/4 [==============================] - 31s 5s/step - loss: 0.7528 - accuracy: 0.5225
Epoch 2/3
4/4 [==============================] - 21s 5s/step - loss: 0.7113 - accuracy: 0.5135
Epoch 3/3
4/4 [==============================] - 21s 5s/step - loss: 0.6874 - accuracy: 0.5495


In [ ]:
predY = model2.predict(testX)

In [ ]:
predY.ndim

2

In [ ]:
testY.ndim

2

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
#อิมพอร์ต accuracy_score กับ classification_report จาก sklearn.metrics
accuracy_score(testY, predY)
#จากนั้นมาเรียกใช้ฟังก์ชั่น accuracy_score เพื่อดูความแม่นของข้อมูลที่ predict กัน

ValueError: ignored

In [ ]:
print(classification_report(testY.argmax(axis=1) , 
                                    predY.argmax(axis=1) , target_names=lb.classes_))
#เราลองมาดูผลการ classification อย่างละเอียดกัน

ValueError: ignored

In [ ]:
print(classification_report(testY.argmax(axis=1) , 
                                    predY.argmax(axis=1) , target_names=lb.classes_))

In [ ]:
predY[:5]

array([[0.5776474 ],
       [0.59712297],
       [0.45053387],
       [0.5679433 ],
       [0.748017  ]], dtype=float32)

In [ ]:
predY.argmax(axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
testY[:5]

array([[1],
       [0],
       [1],
       [0],
       [1]])

In [ ]:
model2.evaluate(trainX, trainY)

4/4 [==============================] - 30s 6s/step - loss: 0.1073 - accuracy: 1.0000


[0.10726463794708252, 1.0]

In [ ]:
predY = model2.predict(testX)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score()

In [ ]:
accuracy_score(testY, predY)

ValueError: ignored

In [ ]:
print(classification_report(testY, predY, target_names=lb.classes_))

NameError: ignored